<a href="https://colab.research.google.com/github/chunribu/manim-video/blob/main/pccfs/20MusicVideos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install libcairo2-dev ffmpeg libpango1.0-dev \
    # texlive texlive-latex-extra texlive-fonts-extra \
    # texlive-latex-recommended texlive-science tipa
!pip install manim
!pip install IPython --upgrade
exit()

In [ ]:
!unzip assets.zip

In [ ]:
from manim import *

In [ ]:
%%manim -v WARNING --disable_caching M20

from math import cos

config.frame_width = 16
config.frame_height = 9
config.frame_size = (3840,2160)#(2560,1440)#(1920,1080)#(720,720)#(480,480)#(1080,1080)#
config.frame_rate = 60
config.background_color = DARK_GRAY


class M20(ThreeDScene):
    def construct(self):

        logo = SVGMobject('pics/logo.svg')
        logo.height = 1.2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=ORIGIN, buff=0.2)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).set_z_index(999).center()
        # self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))

        self.time=0
        def update_time(dt):
            self.time += dt
        self.add_updater(update_time)

        colors = [
            '#30f2f2',
            '#ffd23f',
            '#ff6f59',
            '#6290c3',
            '#a0a4b8',
            '#ff9b85',
            '#ffd97d',
            '#aaf683',
            '#60d394',
            '#ee6055',
            '#f2d0a9',
            '#f1e3d3',
            '#99c1b9',
            '#8e7dbe',
            '#00bbf9',
            '#00f5d4',
        ]
        cycle_time=2
        trans_time=.35
        dot_radius = .25
        total_num = 12
        cur_num = 1

        ring = Torus(3.5, .2, resolution=32, fill_color=BLUE, fill_opacity=1, stroke_width=0, checkerboard_colors=None)
        base = Prism(dimensions=[3, .75, .75], fill_color=BLUE)
        brand.scale_to_fit_width(2.5).rotate(PI/2, axis=RIGHT)
        hub = Circle(radius=dot_radius).set_stroke(WHITE,4,.8).set_z_index(1)
        line = Line(OUT*3, IN*3).set_stroke(WHITE,4,.5).set_z_index(0)
        dot = Dot3D(radius=dot_radius, resolution=(12,12), checkerboard_colors=None).set_opacity(1).set_z_index(2)

        def gen_lines(line_num):
            lines = [
                line.copy().rotate(PI/line_num*i, axis=UP)
                if i < line_num else
                line.copy().set_opacity(0.)
                for i in range(total_num)
            ]
            return VGroup(*lines)

        lines = gen_lines(cur_num)
        # def init_proportion(i):
        #     return (1-cos(i/cur_num*PI))/2
        def get_proportion(i):
            t = self.time%cycle_time + i/cur_num
            p = t if t<=1 else 2-t
            return (1-cos(p*PI))/2
        dots = []
        for i in range(total_num):
            dots.append(dot.copy().set_color(colors[i]))
            dots[i]._i = i
            dots[i]._c = colors[i]
            dots[i]._p = get_proportion(i)
            dots[i]._t = 0
        def update_dot(dot, dt):
            dot._t += dt
            dot.move_to(
                lines[dot._i].point_from_proportion(dot._p)
            )
            if dot._i >= cur_num:
                dot.set_opacity(0)
            else:
                dot.set_opacity(1)
                next_p = get_proportion(dot._i)
                if (next_p>=.5 and dot._p<=.5) or (next_p<=.5 and dot._p>=.5) and dot._t>=.5:
                    self.add_sound(f'kalimba_f/{13-dot._i}', -.06, -6)
                    watermark.set_color(dot._c)
                    dot._t = 0
                dot._p = next_p
        for i in range(total_num):
            dots[i].add_updater(update_dot)
            dots[i].update(0)

        ring.rotate(PI/2, axis=RIGHT)
        base.next_to(ring, IN, buff=0)
        brand.next_to(base, DOWN, buff=0)
        hub.rotate(PI/2,axis=RIGHT)
        statics = VGroup(ring, base, brand, hub)
        self.add(statics, lines, *dots)

        self.set_camera_orientation(phi=90*DEGREES, theta=-90*DEGREES, gamma=0*DEGREES, focal_distance=100)

        for i in range(total_num):
            new_lines = gen_lines(i+1)
            self.play(ReplacementTransform(lines, new_lines, run_time=trans_time, rate_func=linear, suspend_mobject_updating=False))
            cur_num = i+1
            lines = new_lines
            t = 2*cycle_time-trans_time if i>0 else cycle_time-trans_time
            if cur_num == 7:
                self.move_camera(phi=45*DEGREES, theta=0*DEGREES, gamma=0*DEGREES, run_time=t, rate_func=linear)
                continue
            elif cur_num == 8:
                self.move_camera(phi=60*DEGREES, theta=90*DEGREES, gamma=0*DEGREES, run_time=t, rate_func=linear)
                continue
            elif cur_num == 9:
                self.begin_ambient_camera_rotation(rate=TAU/30, about='theta')
                self.wait(t)
            else:
                self.wait(t)

        self.wait(10)
        self.stop_ambient_camera_rotation()
        self.set_camera_orientation(phi=90*DEGREES, theta=-90*DEGREES, gamma=0*DEGREES, focal_distance=20)
        self.wait(5)

        cur_num = 4
        new_lines = gen_lines(4)
        self.play(ReplacementTransform(lines, new_lines, run_time=1, rate_func=linear, suspend_mobject_updating=False))
        lines = new_lines
        self.wait(10)


In [ ]:
%%manim -v WARNING --disable_caching M19

config.frame_width = 9
config.frame_height = 9
config.frame_size = (1080,1080)#(720,720)#(480,480)#(1920,1080)#(2560,1440)#
config.frame_rate = 60

class Box(Square):
    def __init__(self, side_length: float = 2, innercolor=BLACK, **kwargs):
        super().__init__(side_length, **kwargs)
        self._c = innercolor
        self.set_fill(self._c, 0)
        self.set_stroke(self._c, 12, 1)
        self.change_color = False
        self.idx = 0
        self._grad_t = 60
        self._gradient_colors = color_gradient([WHITE, self._c, BLACK], self._grad_t)
        self.add_updater(self._updater)

    def _updater(self, m, dt):
        if self.change_color == True:
            self.idx += dt*10000/self._grad_t
            if self.idx >= self._grad_t-1:
                self.change_color = False
                self.idx = self._grad_t-1
            m.set_fill(self._gradient_colors[int(self.idx)], 1)

    def _flash(self):
        self.change_color = True
        self.idx = 0
        self.rotate(-PI/18)


class M19(Scene):
    def construct(self):

        self.time=0
        def update_time(dt):
            self.time += dt
        self.add_updater(update_time)

        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).set_z_index(999).center()
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))

        n = 12
        widths = np.linspace(6, .5, n)
        c_ = ['#00dbde','#fc00ff',]
        colors = color_gradient(c_, n)
        t_list = np.linspace(6, 2, n)

        boxes = []
        for i in range(n):
            boxes.append(Box(widths[i], colors[i]))
            boxes[i]._i = i
            boxes[i]._t = t_list[i]
            boxes[i]._p = 0
        def update_box(box, dt):
            next_p = self.time%box._t/box._t
            if box._p > next_p:
                self.add_sound(f'kalimba_f/{box._i}', 0, -6)
                watermark.set_color(box._c)
                box._flash()
            box._p = next_p
        for i in range(n):
            boxes[i].add_updater(update_box)

        self.play(FadeIn(VGroup(*boxes), run_time=1, lag_ratio=1))
        for _ in range(60):
            self.wait(60)
        # self.wait(13)

        self.remove(*boxes)
        self.play(brand.animate(run_time=.5).scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching M18

config.frame_width = 16
config.frame_height = 9
config.frame_size = (1920,1080)#(854,480)#(1080,1080)#(2560,1440)#(1280,720)#
config.frame_rate = 60


class M18(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).set_z_index(999).center()
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))

        self.time=0
        def update_time(dt):
            self.time += dt
        self.add_updater(update_time)

        n = 12
        c_ = [RED_D, GOLD, TEAL_D, BLUE_D]
        colors = color_gradient(c_, n)
        d_radius = .2
        t_list = np.linspace(2, 6, n)
        self.n_lines = 1

        upleft, upmid, upright = [-8,4,0], [0,4,0], [8,4,0]
        hub  = Circle(d_radius+.02).move_to(upmid).set_fill(WHITE,.7).set_stroke(WHITE,2,1)
        hub.add_updater(lambda _,dt:None)
        border = ArcBetweenPoints(upleft,upright,PI)
        line = Line(upleft,upright)
        # right_point = lambda i: border.point_from_proportion(.5 + .5/self.n_lines*i)
        # left_point  = lambda i: border.point_from_proportion(.5 - .5/self.n_lines*i)

        def get_lines():
            return VGroup(*[
                Line(upmid, border.point_from_proportion(1 - 1/(self.n_lines+1)*(self.n_lines-i)))
                # VMobject().set_points_as_corners([right_point(i), upmid, left_point(i)])
                if i<self.n_lines else Line(upmid,upright).set_opacity(0)
                for i in range(n)
            ])
        lines = get_lines()

        dots = []
        for i in range(n):
            dots.append(
                Dot(radius=d_radius, color=colors[i]).set_stroke(WHITE, 2)
            )
            dots[i]._p = 1
            dots[i]._i = i
            dots[i]._t = t_list[i]

        def t2p(t):
            perc = self.time % t / t
            p = 1-2*perc #if perc<.5 else 2*perc-1
            return p
        def update_dot(d, dt):
            if d._i<self.n_lines:
                d.set_opacity(1)
            else:
                d.set_opacity(0)
            d.move_to(lines[d._i].point_from_proportion(abs(d._p)))
            next_p = t2p(d._t)
            if d._i<self.n_lines:
                if d._p>=0 and next_p<=0:
                    # pass
                    self.add_sound(f'kalimba_f/{12-d._i}', 0, -6)
                    hub.set_fill(color=colors[d._i])
                    watermark.set_color(colors[d._i])
            d._p = next_p
        for d in dots[::-1]:
            d.add_updater(update_dot)
            d.update(0)

        # def get_links():
        #     return VGroup(*[
        #         Line(ORIGIN, dot.get_center()).set_stroke(opacity=.6)
        #         for dot in dots
        #     ])
        # links = always_redraw(get_links)

        self.n_lines = n
        new_lines = get_lines()
        self.play(ReplacementTransform(lines, new_lines, run_time=.5, rate_func=linear, suspend_mobject_updating=False))
        self.add(line, border, hub, *dots)
        lines = new_lines
        for _ in range(9): self.wait(60)

        for i in range(n-1):
            self.n_lines -= 1
            new_lines = get_lines()
            self.play(ReplacementTransform(lines, new_lines, run_time=.3, rate_func=linear, suspend_mobject_updating=False))
            self.add(hub, *dots, border)
            lines = new_lines
            self.wait(5)
        # self.wait(30)
        # for i in range(n-1):
        #     self.n_lines -= 1
        #     new_lines = get_lines()
        #     self.play(ReplacementTransform(lines, new_lines, run_time=.1, suspend_mobject_updating=False))
        #     self.add(hub, *dots, border)
        #     lines = new_lines
        #     self.wait(2.15 if self.n_lines==1 else 5)

        self.remove(*dots, hub, lines)

        self.play(brand.animate(run_time=.5).scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching M17


config.frame_width = 9
config.frame_height = 9
config.frame_size = (480,480)#(2560,1440)#(1280,720)#(1920,1080)#(1080,1920)#
config.frame_rate = 15

class M17(Scene):
    def construct(self):
        # logo = SVGMobject('pics/logo.svg')
        # logo.height = 2
        # watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        # watermark.add_updater(lambda _, dt: None)
        # brand = VGroup(logo, watermark).center()
        # self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))
        # self.add_sound('vibraphone_c/0.wav', -.1, -10)

        self.time=0
        def update_time(dt):
            self.time += dt
        self.add_updater(update_time)

        n = 8
        c_ = [RED_D, GOLD, TEAL_D, BLUE_D]
        colors = color_gradient(c_, n)
        d_radius = .2
        t_list = np.linspace(4, 1, n)
        n_lines = 1

        hub = Dot(radius=d_radius).set_opacity(.9)
        line = Line(ORIGIN, UP*3.5)
        border = Circle(3.5 + d_radius + .05).set_stroke(WHITE, 10, 1)
        # border.add_updater(lambda _,dt:None)

        def get_lines():
            return VGroup(*[
                line.copy().rotate(TAU/n_lines*i, about_point=ORIGIN)
                if i<n_lines else line.copy().set_opacity(0)
                for i in range(n)
            ])
        lines = get_lines()

        dots = []
        for i in range(n):
            dots.append(
                Dot(radius=d_radius, color=colors[i]).set_stroke(WHITE, 2)
            )
            dots[i]._p = 1
            dots[i]._i = i
            dots[i]._t = t_list[i]

        def t2p(t):
            perc = self.time % t / t
            p = 1-2*perc if perc<.5 else 2*perc-1
            return p
        def update_dot(d, dt):
            if d._i<n_lines:
                d.set_opacity(1)
            else:
                d.set_opacity(0)
            d.move_to(lines[d._i].point_from_proportion(d._p))
            next_p = t2p(d._t)
            if d._i<n_lines:
                if (d._p<0 and next_p>=0) or (d._p>0 and next_p<=0):
                    pass
                    # self.add_sound(f'kalimba_f/{15-d._i}.wav', 0, -6)
                    # border.set_stroke(color=colors[d._i])
                    # watermark.set_color(colors[d._i])
            d._p = next_p
        for d in dots[::-1]:
            d.add_updater(update_dot)
            d.update(0)

        # def get_links():
        #     return VGroup(*[
        #         Line(ORIGIN, dot.get_center()).set_stroke(opacity=.6)
        #         for dot in dots
        #     ])
        # links = always_redraw(get_links)

        for i in range(n-1):
            n_lines = i+2
            new_lines = get_lines()
            self.play(ReplacementTransform(lines, new_lines, run_time=.1, suspend_mobject_updating=False))
            self.add(hub, *dots, border)
            lines = new_lines
            self.wait(4)
        self.wait(10)

        self.remove(*dots)

        # self.add(brand)
        # self.play(brand.animate(run_time=.5).scale(2).center())
        # self.wait()

In [ ]:
%%manim -v WARNING --disable_caching M16


config.frame_width = 16
config.frame_height = 9
config.frame_size = (2560,1440)#(1280,720)#(1920,1080)#(1080,1920)#(480,480)#
config.frame_rate = 60

class M16(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))
        # self.add_sound('vibraphone_c/0.wav', -.1, -10)

        # self.time=0
        # def update_time(dt):
        #     self.time += dt
        # self.add_updater(update_time)


        dr = .2
        colors = [RED, BLUE, YELLOW, TEAL, PINK]
        n_edges = [2,3,5,7]
        N = len(n_edges)
        op = .4
        T = 2
        vols = [-3,-3,-4,-12]
        sounds = ['9','7','5','0']
        sounds = ['kalimba_f/'+s for s in sounds]
        trs = np.linspace(3,4,N)

        circs = [
            Circle(radius=r).rotate(PI/2)
            for r in trs
        ]

        ps = [
            [i/n for i in range(n)]
            for n in n_edges
        ]
        vs = [
            [circs[i].point_from_proportion(p) for p in ps[i]]
            for i in range(N)
        ]
        geoms = [Polygon(*v).set_stroke(colors[i],14,op) for i,v in enumerate(vs)]
        for g in geoms:
            g.add_updater(lambda _,dt:None)


        dots = []
        for i in range(N):
            dots.append(
                Dot([0,3.5,0], dr*1.5, color=colors[i]).set_opacity(1)
            )
            dots[-1]._i = i
            dots[-1]._n = n_edges[i]
            dots[-1]._ps = ps[i]
            dots[-1]._p = .9999
            dots[-1]._g = geoms[i]
            dots[-1]._on = False

        def update_dot(d, dt):
            if d._on:
                d.set_opacity(1)
                strop = d._g.get_stroke_opacity()
                if strop > op:
                    d._g.set_stroke(opacity=strop-dt*2)
                else:
                    d._g.set_stroke(opacity=op)

                d.move_to(d._g.point_from_proportion(d._p))
                next_p = (d._p + dt/T) % 1
                for p in d._ps:
                    if (d._p<=p and next_p>=p) or (next_p < d._p):
                        self.add_sound(sounds[d._i], 0, vols[d._i])
                        geoms[d._i].set_stroke(opacity=1)
                        watermark.set_color(colors[d._i])
                        break
                d._p = next_p
            else:
                d.set_opacity(0)
                d._g.set_stroke(opacity=0)
                d._p = .9999
        for d in dots[::-1]:
            d.add_updater(update_dot)


        self.add(*geoms[::-1], *dots[::-1])
        dots[0]._on = True
        self.wait(T)
        dots[0]._on = False
        dots[1]._on = True
        self.wait(T)
        dots[1]._on = False
        dots[2]._on = True
        self.wait(T)
        dots[2]._on = False
        dots[3]._on = True
        self.wait(T)
        dots[3]._on = False
        # dots[4]._on = True
        # self.wait(T)
        # dots[4]._on = False
        dots[3].update(0)

        def play(i0,i1,i2,i3):
            dots[i0]._on = True
            self.wait(T)
            dots[i1]._on = True
            self.wait(T*2)
            dots[i2]._on = True
            self.wait(T*2)
            dots[i3]._on = True
            # self.wait(T*2)
            # dots[i4]._on = True

            self.wait(T*5)

            dots[i0]._on = False
            self.wait(T*2)
            dots[i1]._on = False
            self.wait(T*2)
            # dots[i2]._on = False
            # self.wait(T*2)
            dots[i2]._on = False
            self.wait(T+.07)
            dots[i3]._on = False
            dots[i3].update(0)

        for i in [
            [3,2,1,0],
            [0,1,2,3],
        ]:
            play(*i)

        self.add(brand)
        self.play(brand.animate(run_time=.5).scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching M14


config.frame_width = 9
config.frame_height = 9
config.frame_size = (480,480)#(1920,1080)#(2560,1440)#(1080,1920)#
config.frame_rate = 30


class M14(Scene):
    def construct(self):
        def play(n_edges=[3,4,12]):
            dr = .16
            colors = ['#ff7473','#ffc952','#47b8e0']
            # n_edges = [3,4,6,12]
            N = len(n_edges)
            op = .4
            T = 2
            vols = [0,0,-6]
            sounds = ['kick_drum','open_conga','low_bongo',]
            sounds = ['drumkit/'+s for s in sounds]
            trs = np.linspace(2.5,3.5,N)

            circs = [
                Circle(radius=r).rotate(PI/2)
                for r in trs
            ]

            ps = [
                [i/n for i in range(n)]
                for n in n_edges
            ]
            vs = [
                [circs[i].point_from_proportion(p) for p in ps[i]]
                for i in range(N)
            ]
            geoms = [Polygon(*v).set_stroke(colors[i],14,op) for i,v in enumerate(vs)]
            for g in geoms:
                g.add_updater(lambda _,dt:None)

            dots = []
            for i in range(N):
                dots.append(
                    Dot([0,3.5,0], dr*1.5, color=colors[i]).set_opacity(1)
                )
                dots[-1]._i = i
                dots[-1]._n = n_edges[i]
                dots[-1]._ps = ps[i]
                dots[-1]._p = .9999
                dots[-1]._g = geoms[i]
                dots[-1]._on = False

            def update_dot(d, dt):
                if d._on:
                    d.set_opacity(1)
                    strop = d._g.get_stroke_opacity()
                    if strop > op:
                        d._g.set_stroke(opacity=strop-dt*2)
                    else:
                        d._g.set_stroke(opacity=op)

                    d.move_to(d._g.point_from_proportion(d._p))
                    next_p = (d._p + dt/T) % 1
                    for p in d._ps:
                        if (d._p<=p and next_p>=p) or (next_p < d._p):
                            self.add_sound(sounds[d._i], 0, vols[d._i])
                            geoms[d._i].set_stroke(opacity=1)
                            watermark.set_color(colors[d._i])
                            break
                    d._p = next_p
                else:
                    d.set_opacity(0)
                    d._g.set_stroke(opacity=0)
                    d._p = .9999
            for d in dots[::-1]:
                d.add_updater(update_dot)

            def play(i0,i1,i2):
                dots[i0]._on = True
                self.wait(T)
                dots[i1]._on = True
                self.wait(T*2)
                dots[i2]._on = True

                self.wait(T*7)

                dots[i0]._on = False
                self.wait(T*2)
                dots[i1]._on = False
                self.wait(T+.07)
                dots[i2]._on = False
                dots[i2].update(0)

            self.add(*geoms[::-1], *dots[::-1])
            dots[0]._on = True
            self.wait(T)
            dots[0]._on = False
            dots[1]._on = True
            self.wait(T)
            dots[1]._on = False
            dots[2]._on = True
            self.wait(T)
            dots[2]._on = False
            dots[2].update(0)

            for i in [
                [2, 1, 0],
                # [0, 2, 1],
                # [1, 0, 2],
            ]:
                play(*i)
            self.remove(*geoms[::-1], *dots[::-1])


        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))

        for i in [
            # [3,4,12],
            [4,5,6],
        ]:
            play(i)

        self.play(brand.animate(run_time=.5).scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching M13

import math
PI = math.pi
PID = PI/2
DPI = PI*2

def argument(x,y):
    if x!=0:
        alph = math.atan(y/x)
        if x<0:
            alph += PI
    else:
        alph = PID
        if y<0:
            alph = PI*1.5
    return alph

def _next(pos, alpha, lam):
    c0 = math.cos(alpha)
    s0 = math.sin(alpha)
    lam2 = lam**2

    a = c0**2 + lam2*s0**2
    b = pos[0]*c0 + lam2*pos[1]*s0
    c = pos[0]**2 + lam2*pos[1]**2 - lam2

    t = (-b+math.sqrt(b**2 - a*c))/a
    x1 = pos[0] + t*c0
    y1 = pos[1] + t*s0

    # param1 = argument(x1/lam, y1)
    # while param1<0: param1 += DPI
    # while param1>DPI: param1 -= DPI

    theta = argument(-lam*y1, x1/lam)
    param2 = theta - alpha
    while param2<0: param2 += DPI
    while param2>DPI: param2 -= DPI
    alpha = theta + param2

    return (x1,y1), alpha


# lam = 5/4
# pos = (-.5,0)
# alpha = PI/6

def simulate(pos=(0,0), alpha=PI/2, times=10, lam=5/4):
    points = []
    for i in range(times):
        # print(pos, alpha/PI)
        points.append(pos)
        pos, alpha = _next(pos, alpha, lam)
        # pos = (x,y)
        # x = lam * math.cos(param1)
        # y = math.sin(param1)
        # theta = argument(-lam*y, x/lam)
        # alpha = theta + param2
    return points

# print(simulate())

def vec3d(ps):
    n = len(ps)
    ps = np.array(ps)
    zeros = np.zeros((n,1))
    return np.c_[ps, zeros]


config.frame_width = 16
config.frame_height = 9
config.frame_size = (1920,1080)#(854,480)#(2560,1440)#
config.frame_rate = 60

class M13(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="Serif", weight=BOLD).scale(.6).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _,dt:None)
        brand = VGroup(logo, watermark).center().set_z_index(999)
        self.add(brand)
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5,UP,buff=0))

        n = 3*4
        n25, n50, n75 = int(n/4), int(n/2), int(n*3/4)
        dr = .2
        c_ = [RED_D, RED, YELLOW, TEAL, BLUE_D]
        colors = color_gradient(c_, int(n))
        pos_0 = [(1.0, i) for i in np.linspace(-.25, .25, n)]
        angles = [PI*10/6 for i in range(n)]
        t1, t2, t3, t4 = 20, 20, 20, 25
        t_list = [t1,t2,t3,t4]

        s = 4
        w, h = 10, 8
        d = 3

        cld_time1 = 20 # speed x1
        cld_time2 = 100 # x5
        cld_time3 = 200 # x10
        cld_time4 = 400 # x15
        cld_time_list = [cld_time1, cld_time2, cld_time3, cld_time4]
        speed_list = [1,5,10,15]

        def get_points(pos=(-d/s,0), alpha=PI/2, times=243, lam=w/h):
            return simulate(pos, alpha, times, lam)
            return vec3d(ps[:2])*s, vec3d(ps[2:])*s
        def calc_dist(ps):
            n = len(ps)
            d = 0
            for i in range(n-1):
                d += np.linalg.norm(ps[i]-ps[i+1])
            return d

        def get_path(ps, color=WHITE):
            a, b = ps[:2], ps[2:]
            path = VMobject().set_stroke(color,2,.4).set_points_as_corners(a)
            if len(b): path.add_points_as_corners(b)
            return path

        def play(T, cld_time, speed):
            ps_list = [get_points(pos_0[i], angles[i], times=cld_time) for i in range(n)]
            di_list = [calc_dist(ps) for ps in np.array(ps_list)]

            paths, proportions = [], []
            for i in range(n):
                ps = vec3d(ps_list[i]) * s
                # b_ = np.r_[b, np.array([[-d,0,0]])]
                paths.append(get_path(ps, color=colors[i]))
                proportions.append([paths[-1].proportion_from_point(p) for p in ps[1:] ])

            def update_dot(d, dt):
                if d._p>1:
                    d._p=1
                d.move_to(d._t.point_from_proportion(d._p))
                next_p = d._p + dt / d._T
                r = d._r[d._ri] if d._p<1 else 0
                if d._i<n and d._p<=r and next_p>=r:
                    ########## add sound
                    self.add_sound(f'kalimba_f/{n-1-d._i}', -.05, -6)
                    elli.set_color(d._c)
                    watermark.set_color(d._c)
                    d._ri += 1
                d._p = next_p

            dots = []
            for i in range(n):
                dots.append(Dot(radius=dr, color=colors[i]))#.set_stroke(WHITE,2,1)
                dots[-1]._i = i
                dots[-1]._c = colors[i]
                dots[-1]._t = paths[i]
                dots[-1]._p = 0
                dots[-1]._r = proportions[i]
                dots[-1]._ri = 0
                dots[-1]._trace = TracedPath(dots[-1].get_center, stroke_opacity=.4, stroke_width=2, stroke_color=colors[i])
                dots[-1]._T = T #/min(di_list)*di_list[i]
            for d in dots[::-1]:
                d.add_updater(update_dot)
                d.update(0)

            elli = Ellipse(height=h+dr*2, width=w+dr*2, color=WHITE, stroke_width=10)
            elli.add_updater(lambda _,dt:None)

            speed_text = Text(f'Speed x{speed}').next_to(UL*4+LEFT*3, RIGHT, buff=0)

            self.add(*[d._trace for d in dots], *dots, elli, speed_text)#
            self.wait(T)
            self.remove(*[d._trace for d in dots], *dots, elli, speed_text)
        for T, cld_time, speed in zip(t_list, cld_time_list, speed_list):
            play(T, cld_time, speed)


In [ ]:
%%manim -v WARNING --disable_caching M12

import math

PI = math.pi
PID = PI/2
DPI = PI*2

def argument(x,y):
    if x!=0:
        alph = math.atan(y/x)
        if x<0:
            alph += PI
    else:
        alph = PID
        if y<0:
            alph = PI*1.5
    return alph

def _next(pos, alpha, lam):
    c0 = math.cos(alpha)
    s0 = math.sin(alpha)
    lam2 = lam**2

    a = c0**2 + lam2*s0**2
    b = pos[0]*c0 + lam2*pos[1]*s0
    c = pos[0]**2 + lam2*pos[1]**2 - lam2

    t = (-b+math.sqrt(b**2 - a*c))/a
    x1 = pos[0] + t*c0
    y1 = pos[1] + t*s0

    # param1 = argument(x1/lam, y1)
    # while param1<0: param1 += DPI
    # while param1>DPI: param1 -= DPI

    theta = argument(-lam*y1, x1/lam)
    param2 = theta - alpha
    while param2<0: param2 += DPI
    while param2>DPI: param2 -= DPI
    alpha = theta + param2

    return (x1,y1), alpha


# lam = 5/4
# pos = (-.5,0)
# alpha = PI/6

def simulate(pos=(0,0), alpha=PI/2, times=10, lam=5/4):
    points = []
    for i in range(times):
        # print(pos, alpha/PI)
        points.append(pos)
        pos, alpha = _next(pos, alpha, lam)
        # pos = (x,y)
        # x = lam * math.cos(param1)
        # y = math.sin(param1)
        # theta = argument(-lam*y, x/lam)
        # alpha = theta + param2
    return points

# print(simulate())

def vec3d(ps):
    n = len(ps)
    ps = np.array(ps)
    zeros = np.zeros((n,1))
    return np.c_[ps, zeros]

# from manim import *

config.frame_width = 16
config.frame_height = 9
config.frame_size = (854,480)#(1920,1080)#(2560,1440)#
config.frame_rate = 15

class M12(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).scale(.6).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _,dt:None)
        brand = VGroup(logo, watermark).center().rotate(-PI/2)
        self.play(Write(brand))
        self.play(brand.animate(run_time=1).scale(.5).next_to(LEFT*8,RIGHT,buff=0))

        n = 14*2
        dr = .1
        c_ = [RED_D, RED, YELLOW, TEAL, BLUE_D]
        colors = color_gradient(c_, int(n/2))
        colors = colors[::-1] + colors
        angles = np.arange(PI/n, TAU, TAU/n)
        T = 22

        s = 4
        w, h = 10, 8
        d = 3

        def get_points(pos=(-d/s,0), alpha=PI/2, times=243, lam=w/h):
            ps = simulate(pos, alpha, times, lam)
            return vec3d(ps[:2])*s, vec3d(ps[2:])*s
        def get_path(a, b, color=WHITE):
            path = VMobject().set_stroke(color,2,.5).set_points_as_corners(a)
            if len(b): path.add_points_as_corners(b)
            return path

        paths, proportions = [], []
        for i in range(n):
            a, b = get_points(alpha=angles[i], times=3)
            b_ = np.r_[b, np.array([[-d,0,0]])]
            paths.append(get_path(a, b_, color=colors[i]))
            proportions.append([paths[-1].proportion_from_point(p) for p in np.r_[a[1:], b] ])

        def update_dot(d, dt):
            if d._p>1:
                d._p=1
            d.move_to(d._t.point_from_proportion(d._p))
            next_p = d._p + dt/T
            for r in d._r:
                if d._i<n/2 and d._p<=r and next_p>=r:
                    ########## add sound
                    self.add_sound(f'kalimba_f/{d._i}', -.05, -8)
                    elli.set_color(d._c)
                    watermark.set_color(d._c)
                    break
            d._p = next_p

        dots = []
        for i in range(n):
            dots.append(Dot(radius=dr, color=colors[i]))#.set_stroke(WHITE,2,1)
            dots[-1]._i = i
            dots[-1]._c = colors[i]
            dots[-1]._t = paths[i]
            dots[-1]._p = 0
            dots[-1]._r = proportions[i]
            dots[-1]._trace = TracedPath(dots[-1].get_center, stroke_opacity=.3, stroke_width=2, stroke_color=colors[i])
        for d in dots[::-1]:
            d.add_updater(update_dot)
            d.update(0)

        elli = Ellipse(height=h+.2, width=w+.2, color=WHITE, stroke_width=8)
        elli.add_updater(lambda _,dt:None)
        d_left = Circle(radius=dr*2, color=WHITE, stroke_width=2).move_to(LEFT*3)
        d_right = d_left.copy().move_to(RIGHT*3)

        pencil = SVGMobject('pics/pencil.svg').set_color(WHITE)
        pencil.height = .5
        pencil._p = 0
        def update_pencil(m, dt):
            m.align_to(elli.point_from_proportion(m._p), DL)
            m._p = m._p + dt/3 if m._p<.99 else 1
        pencil.add_updater(update_pencil)
        pencil.update(0)

        path = VMobject(stroke_width=8)
        path.set_points_as_corners([pencil.get_corner(DL), pencil.get_corner(DL)])
        def update_path(path):
            previous_path = path.copy()
            previous_path.add_points_as_corners([pencil.get_corner(DL)])
            path.become(previous_path)
        path.add_updater(update_path)

        self.wait(.5)
        self.add_sound('sound_effects/pencil.wav', -.1, -5)
        self.add(path, pencil)
        self.wait(3.1)
        self.remove(pencil, path)
        self.add(elli)
        self.play(Create(d_left), Create(d_right), run_time=.5)

        self.add(*[d._trace for d in dots], *dots)
        self.wait(T+.05)

        self.add(brand)
        self.play(brand.animate(run_time=1).scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching M11


config.frame_width = 16
config.frame_height = 9
config.frame_size = (1920,1080)#(2560,1440)#(1080,1920)#(480,480)#
config.frame_rate = 60


class M11(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))
        # self.add_sound('vibraphone_c/0.wav', -.1, -10)

        # self.time=0
        # def update_time(dt):
        #     self.time += dt
        # self.add_updater(update_time)


        dr = .16
        colors = ['#ff7473','#ffc952','#47b8e0',GRAY_B]
        n_edges = [3,4,6,12]
        N = len(n_edges)
        op = .4
        T = 2
        vols = [0,-2,-6,-4]
        sounds = ['kick_drum','open_conga','side_stick','low_bongo',]
        # sounds = ['4','7','10','13'][::-1]
        sounds = ['drumkit/'+s for s in sounds]
        trs = np.linspace(2.5,3.5,N)

        circs = [
            Circle(radius=r).rotate(PI/2)
            for r in trs
        ]

        ps = [
            [i/n for i in range(n)]
            for n in n_edges
        ]
        vs = [
            [circs[i].point_from_proportion(p) for p in ps[i]]
            for i in range(N)
        ]
        geoms = [Polygon(*v).set_stroke(colors[i],14,op) for i,v in enumerate(vs)]
        for g in geoms:
            g.add_updater(lambda _,dt:None)


        dots = []
        for i in range(N):
            dots.append(
                Dot([0,3.5,0], dr*1.5, color=colors[i]).set_opacity(1)
            )
            dots[-1]._i = i
            dots[-1]._n = n_edges[i]
            dots[-1]._ps = ps[i]
            dots[-1]._p = .9999
            dots[-1]._g = geoms[i]
            dots[-1]._on = False

        def update_dot(d, dt):
            if d._on:
                d.set_opacity(1)
                strop = d._g.get_stroke_opacity()
                if strop > op:
                    d._g.set_stroke(opacity=strop-dt*2)
                else:
                    d._g.set_stroke(opacity=op)

                d.move_to(d._g.point_from_proportion(d._p))
                next_p = (d._p + dt/T) % 1
                for p in d._ps:
                    if (d._p<=p and next_p>=p) or (next_p < d._p):
                        self.add_sound(sounds[d._i], 0, vols[d._i])
                        geoms[d._i].set_stroke(opacity=1)
                        watermark.set_color(colors[d._i])
                        break
                d._p = next_p
            else:
                d.set_opacity(0)
                d._g.set_stroke(opacity=0)
                d._p = .9999
        for d in dots[::-1]:
            d.add_updater(update_dot)


        self.add(*geoms[::-1], *dots[::-1])
        dots[0]._on = True
        self.wait(T)
        dots[0]._on = False
        dots[1]._on = True
        self.wait(T)
        dots[1]._on = False
        dots[2]._on = True
        self.wait(T)
        dots[2]._on = False
        dots[3]._on = True
        self.wait(T)
        dots[3]._on = False

        def play(i0,i1,i2,i3):
            dots[i0]._on = True
            self.wait(T)
            dots[i1]._on = True
            self.wait(T*2)
            dots[i2]._on = True
            self.wait(T*2)
            dots[i3]._on = True

            self.wait(T*5)

            dots[i3]._on = False
            self.wait(T*2)
            dots[i2]._on = False
            self.wait(T*2)
            dots[i1]._on = False
            self.wait(T)
            dots[i0]._on = False

        for _ in range(2):
            for i in [
                [1, 0, 2, 3],
                [2, 0, 3, 1],
                [0, 1, 3, 2],
                [0, 2, 3, 1],
                [3, 1, 2, 0],
                [0, 1, 2, 3],
                [3, 0, 2, 1],
                [2, 0, 1, 3],
                [1, 2, 0, 3],
                [1, 2, 3, 0],
                [2, 1, 0, 3],
                [1, 0, 3, 2],
                [0, 2, 1, 3],
                [3, 2, 0, 1],
                [1, 3, 0, 2],
                [2, 3, 1, 0],
                [2, 3, 0, 1],
                [3, 2, 1, 0],
                [0, 3, 2, 1],
                [3, 0, 1, 2],
                [2, 1, 3, 0],
                [3, 1, 0, 2],
                [0, 3, 1, 2],
                [1, 3, 2, 0]
            ]:
                play(*i)

        self.add(brand)
        self.play(brand.animate(run_time=.5).scale(2).center())
        self.wait()

In [ ]:
ijkl = [[i,j,k,l] for i in range(4) for j in range(4) for k in range(4) for l in range(4) if i!=j and i!=k and i!=l and j!=k and j!=l and k!=l ]
random.shuffle(ijkl)
ijkl

In [ ]:
%%manim -v WARNING --disable_caching M10

import numpy as np
import math

config.frame_width = 9
config.frame_height = 9
config.frame_size = (480,480)#(1080,1920)#(2560,1440)
config.frame_rate = 15

class Dot_(Dot):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._o = 0
        self._od = 0
        self.set_fill(opacity=0)
        self.add_updater(self._update_opacity)

    def _update_opacity(self, m, dt):
        if m._o <= 0:
            pass
        elif m._od == 1:
            m._o += dt * 3
        elif m._od == -1:
            m._o -= dt / 2
        o = np.clip(m._o, 0, 1)
        m.set_fill(opacity=o)
        if o >= 1:
            m._od = -1
        if o < 0:
            m._od = 1
    def _set_o(self, o, od=1):
        self._o = o
        self._od = od
        self.set_fill(opacity=1)

class M10(Scene):
    def construct(self):
        # logo = SVGMobject('pics/logo.svg')
        # logo.height = 2
        # watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        # watermark.add_updater(lambda _, dt: None)
        # brand = VGroup(logo, watermark).center()
        # self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))
        # self.add_sound('vibraphone_c/0.wav', -.1, -10)

        self.time=0
        def update_time(dt):
            self.time += dt
        self.add_updater(update_time)


        n = 12
        c_ = [RED_D, GOLD, TEAL, BLUE_D]
        dot_radius = .18 #.07
        angles = np.linspace(PI, -PI, n)
        colors = color_gradient(c_, n)

        from_ = RIGHT*4
        to_ = LEFT*4

        line = Line(UP*4, DOWN*4).set_stroke(WHITE, dot_radius*50, .2)#c_[::-1]

        tracks = [ArcBetweenPoints(from_, to_, angles[i]).set_stroke(colors[i],dot_radius*200,.3) for i in range(n)]

        dots = []
        for i in range(n):
            for j in range(n-i):
                dots.append(Dot_(radius=dot_radius, stroke_width=2, stroke_color=colors[i], color=WHITE))
                dots[-1]._i = i
                dots[-1]._j = j
                dots[-1]._p = (.5+j)/(n-i)
                dots[-1]._t = tracks[i]

        def update_dot(d, dt):
            d.move_to(d._t.point_from_proportion(d._p))
            next_p = (d._p + dt/20) % 1
            if d._p <=.5 and next_p >= .5:
                self.add_sound(f'kalimba_f/{n-d._i}.wav', .05, -8)
                d._set_o(.8)
                line.set_color(colors[d._i])
            d._p = next_p
        for d in dots[::-1]:
            d.add_updater(update_dot)


        self.add(line, *tracks, *dots)
        self.wait(20)

In [ ]:
%%manim -v WARNING --disable_caching M9

import numpy as np
import math

config.frame_width = 9
config.frame_height = 16
config.frame_size = (1080,1920)#(480,480)#(2560,1440)
config.frame_rate = 60


class M9(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))
        self.add_sound('vibraphone_c/0.wav', -.1, -10)

        self.time=0
        def update_time(dt):
            self.time += dt
        self.add_updater(update_time)


        n = 14
        c_ = [RED_D, GOLD, TEAL, BLUE_D]
        dot_radius = .14
        radius_list = np.linspace(1, 3.8, n)
        colors = color_gradient(c_, n)
        x_offset = .25
        left_x, right_x = -x_offset, x_offset

        line1 = Line(ORIGIN+UP*dot_radius, DOWN*4).set_stroke(
            width=(x_offset-dot_radius)*200,
            opacity=.8,
        )
        line2 = Line([-8,-4,0], [8,-4,0])
        lines = VGroup(line1, line2)
        lines.add_updater(lambda _,dt:None)
        hub = Dot(radius=.1, color=BLACK)

        tracks = [
            ArcBetweenPoints(
                [right_x,-r,0],
                [left_x,-r,0],
                angle=TAU-math.asin(right_x/r)*2,
            )
            for r in radius_list
        ]

        def next_p(dot):
            t = (35-dot._i)/42 * self.time % TAU if self.time>0 else 0
            if t < PI/2:
                return math.cos(t) * .5
            elif t < PI:
                return abs(math.cos(t)) * .5
            elif t< PI*1.5:
                return (math.cos(t) * .5) + 1
            else:
                return 1 - (math.cos(t) * .5)
        def update_dot(d, dt):
            d.move_to(tracks[d._i].point_from_proportion(d._p))
            _p = next_p(d)
            if (d._d<0 and _p - d._p > 0) or (d._d>0 and _p-d._p<0):
                ################ add sound ###############
                self.add_sound(f'kalimba_f/{13-d._i}.wav', .05, -6)
                lines.set_color(colors[d._i])
                watermark.set_color(colors[d._i])
                d._d *= -1
            d._p = _p

        dots = []
        for i in range(n):
            dots.append(Dot(radius=dot_radius, color=colors[i]))
            dots[i].move_to([0,radius_list[i],0]).set_stroke(WHITE,2)
            dots[i]._i = i
            dots[i]._p = .5
            dots[i]._d = -1
        for i in range(n):
            dots[n-1-i].add_updater(update_dot)

        def get_links():
            return VGroup(*[
                Line(ORIGIN, dots[i].get_center()).set_stroke(colors[i], 8, .7)
                for i in range(n-1,-1,-1)
            ])
        links = always_redraw(get_links)

        self.add(lines, hub, links, *dots[::-1])
        self.wait(132)
        self.wait(132)

        self.remove(links, *dots)
        self.add(brand)
        self.play(brand.animate(run_time=.5).scale(2).center())
        self.wait()

In [ ]:
# %%manim -v WARNING --disable_caching M8 ############# 再次尝试失败
# from manim import *
# import numpy as np
import pymunk
import pickle


############ GEN CURVE POINTS ##############
vec2d = lambda arr: tuple(arr[:2])
vec3d = lambda arr: arr + (0,)

n_points = 280
_2a = 10
_2b = 8
curve = Ellipse(_2a, _2b)
cps = [vec2d(curve.point_from_proportion(p))
    for p in np.arange(.5/n_points, 1, 1/n_points)]

with open('cps.pkl', 'wb') as f:
    pickle.dump(cps, f)


################## SPACE ###################
space = pymunk.Space()
space.gravity = 0,0

################# DYNEMIC ##################
def new_dot(position=(-3,0), velocity=(0,2), filter=(0b10, 0b01), space=space):
    dot = pymunk.Body()
    dot.position = position
    dot.velocity = velocity
    dot_shape = pymunk.Circle(dot, .0001)
    dot_shape.density = 1
    dot_shape.elasticity = 1
    dot_shape.friction = 0
    dot_shape.filter = pymunk.ShapeFilter(categories=filter[0], mask=filter[1])
    space.add(dot, dot_shape)
    return dot

n_dots = 14
dots = []
circ = Circle(2)
for i in np.arange(.5/n_dots, 1, 1/n_dots):
    v = circ.point_from_proportion(i)
    dots.append(new_dot(velocity=vec2d(v)))


################## STATIC #################
with open('cps.pkl', 'rb') as f:
    cps = pickle.load(f)
n_points = len(cps)
for i in range(n_points):
    seg = pymunk.Segment(space.static_body, cps[i], cps[(i+1) % n_points], .001)
    seg.elasticity = 1
    seg.friction = 0
    space.add(seg)


################# SIMULATE #################
dt = 1/1000
seconds = 15
total_frames = int(seconds / dt)
pt = {i:{} for i in range(n_dots)}
for i in range(total_frames+1):
    for idx in range(n_dots):
        pt[idx][dt*i] = tuple(dots[idx].position)
    space.step(dt)

with open('pt.pkl', 'wb') as f:
    pickle.dump(pt, f)


################ ANIMATE ###################
config.frame_width=16
config.frame_height=9

config.frame_size=(640,360)
config.frame_rate=30

def t2p(time, ipt):
    nearest_t = 0
    for t in ipt:
        if t >= time:
            nearest_t = t
            break
    return vec3d(ipt[nearest_t])

class M8(Scene):
    def construct(self):
        self.time=0
        def update_time(dt):
            self.time+=dt
        self.add_updater(update_time)

        dots = []
        for i in range(n_dots):
            dots.append(Dot(radius=.05))
            dots[i]._i = i
            dots[i].add_updater(lambda d,dt: d.move_to(t2p(self.time, pt[d._i])))
            dots[i].update(0)

        border = VMobject().set_points_as_corners([vec3d(p) for p in cps])

        self.add(border, *dots)
        self.wait(15)

In [ ]:
%%manim -v WARNING --disable_caching Music7
# from manim import *
# import numpy as np
import math

config.frame_width=9
config.frame_height=16

config.frame_size=(480,480)
config.frame_rate=30

config.frame_size=(1080,1920)
config.frame_rate=60

# config.frame_size=(2560,1440)
# config.frame_rate=60

class Music7(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))
        # self.wait()

        n = 20
        c_ = [RED_D, GOLD, TEAL_D, BLUE_D]
        colors = color_gradient(c_, int(n/2)) + color_gradient(c_[::-1], int(n/2))
        d_radius = .5
        # colors = [
        #     '#30f2f2',
        #     '#ffd23f',
        #     '#ff6f59',
        #     '#6290c3',
        #     '#a0a4b8',
        #     '#ff9b85',
        #     '#ffd97d',
        #     '#aaf683',
        #     '#60d394',
        #     '#ee6055',
        #     '#f2d0a9',
        #     '#f1e3d3',
        #     '#99c1b9',
        #     '#8e7dbe',
        #     '#00bbf9',
        #     '#00f5d4',
        # ]

        line = Line(ORIGIN, UP*3.5)
        border = Circle(3.5 + d_radius + .05).set_stroke(WHITE, 10, 1).set_z_index(999)
        border.add_updater(lambda _,dt:None)

        lines = [line.copy().rotate(TAU/n*i, about_point=ORIGIN) for i in range(n)]
        dots = []
        for i in range(n):
            dots.append(
                Dot(radius=d_radius, color=colors[i])\
                .set_stroke(WHITE, 2)\
                .move_to(lines[i].points[-1])
            )
            dots[i]._p = 1
            dots[i]._i = i
            dots[i]._d = -1

        self.time=0
        def timer(dt):
            self.time+=dt
        self.add_updater(timer)

        def tfunc(t):
            t = 0 if t<0 else t
            # return math.cos(t)**2
            t = t%2
            return 1-t if t<1 else t-1
        def time2perc(m):
            return tfunc((12+m._i) *self.time / 40)
            # return tfunc((12+m._i) *self.time / 18)
            # return tfunc((20+m._i) *self.time / 66)
        def update_dot(d, dt):
            d.scale_to_fit_width(d_radius*2*d._p +.01)
            d.move_to(lines[d._i].point_from_proportion(d._p))
            next_p = time2perc(d)
            if (next_p - d._p) * d._d < 0:
                next_d = -d._d
            else: next_d = d._d
            if d._d>0 and next_d<0:
                # self.add_sound(f'xphone/{d._i+1 if d._i<10 else 19-d._i+1}.wav', -.01, -10)
                # self.add_sound(f'vibraphone_c/{9-d._i+2 if d._i<10 else 9-(19-d._i)+2}.wav', -.1, -10)
                # self.add_sound(f'kalimba_f/{14-d._i if d._i<15 else (d._i-15)+1}.wav', -.05, -10)
                # self.add_sound(f'kalimba_f/{15-d._i}.wav', -.03, -10)
                self.add_sound(f'xphone/{19-d._i}.wav', -.05, -10)
                border.set_stroke(color=colors[d._i])
                watermark.set_color(colors[d._i])
            d._d = next_d
            d._p = next_p
        for d in dots[::-1]:
            d.add_updater(update_dot)

        def get_links():
            return VGroup(*[
                Line(ORIGIN, dot.get_center()).set_stroke(opacity=.6)
                for dot in dots
            ])
        links = always_redraw(get_links)

        self.add_sound('vibraphone_c/0.wav', -.1, -10)
        self.add(links, *dots, border)
        # self.wait(56.72)
        # self.wait(69.32)
        # self.update_self(79)
        # self.wait(80.1)
        self.wait(40.1)

        self.remove(*dots, links)
        self.add(brand)
        self.play(brand.animate(run_time=.5).scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching Music6
# from manim import *
# import numpy as np
import math

config.frame_width=9
config.frame_height=9
config.frame_size=(480,480)
config.frame_rate=15
# config.frame_size=(2560,1440)
# config.frame_rate=60

class Music6(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        brand.scale(.5).next_to(DOWN*4.5, UP, buff=0)
        self.add(brand)

        cycle=2
        N = 14
        colors = [
            '#30f2f2',
            '#ffd23f',
            '#ff6f59',
            '#6290c3',
            '#a0a4b8',
            '#ff9b85',
            '#ffd97d',
            '#aaf683',
            '#60d394',
            '#ee6055',
            '#f2d0a9',
            '#f1e3d3',
            '#99c1b9',
            '#8e7dbe',
            '#00bbf9',
            '#00f5d4',
        ]
        hub = Dot(radius=.25).set_fill(WHITE,.9)
        line = Line(UP*4, DOWN*4)

        def new_dots(n=16, d_radius = .25, colors=colors):
            border = Circle(4,WHITE)
            lines = [line.copy().set_stroke(WHITE,4,.8).rotate(PI/n*i) for i in range(n)]
            dots = []
            for i in range(n):
                dots.append(Dot(radius=d_radius, color=colors[i]).set_stroke(WHITE, 2))
                p = i/n
                dots[i]._p = p
                dots[i]._i = i
                dots[i]._dp = None
            def update_dot(d, dt):
                p = d._p%2
                p = p if p<=1 else 2-p
                p = (1-math.cos(p*PI))/2
                pos = lines[d._i].point_from_proportion(p)
                d.move_to(pos)
                if d._dp:
                    if (d._dp - .5) * (p - .5) < 0:
                        self.add_sound(f'kalimba_f/{N-1-d._i}.wav', -.1, -10)
                        watermark.set_color(colors[d._i])
                        d._dp = p
                else: d._dp = p
                _p = d._p + dt/cycle*2
                d._p = _p
            for d in dots:
                d.add_updater(update_dot)
                d.update(0)
            outer = Polygon(*[l.points[i] for i in [0,-1] for l in lines]).set_stroke(WHITE, 10, .8)
            outer.add_updater(lambda _,dt:None)
            return lines, dots, outer

        _outer = line.copy()
        _lines, _dots = None, None
        _t = .1
        for n in range(1,N+1):
            lines, dots, outer = new_dots(n)
            if not _lines:
                self.play(
                    *[FadeIn(l, suspend_mobject_updating=False) for l in lines],
                    FadeIn(hub, suspend_mobject_updating=False),
                    *[FadeIn(d, suspend_mobject_updating=False) for d in dots],
                    run_time=_t,
                    rate_func=linear,
                )
            else:
                self.play(
                    ReplacementTransform(_outer, outer, suspend_mobject_updating=False),
                    *[ReplacementTransform(l, lines[i], suspend_mobject_updating=False) for i, l in enumerate(_lines)],
                    Write(lines[-1], suspend_mobject_updating=False),
                    Write(hub, suspend_mobject_updating=False),
                    *[ReplacementTransform(d, dots[i], suspend_mobject_updating=False) for i, d in enumerate(_dots)],
                    Write(dots[-1], suspend_mobject_updating=False),
                    run_time=_t,
                    rate_func=linear,
                )
            if n in range(2,10):
                self.wait(2*cycle-_t)
            else:
                self.wait(1*cycle-_t)
            _outer = outer
            _lines = lines
            _dots = dots
        # for i in range(4):
        #     cycle -= .1
        #     self.wait(cycle/2)
        # self.wait(cycle*2+_t)

        for n in range(N-1, 0, -1):
            lines, dots, outer = new_dots(n)
            self.play(
                ReplacementTransform(_outer, outer, suspend_mobject_updating=False),
                *[ReplacementTransform(_lines[i], l, suspend_mobject_updating=False) for i, l in enumerate(lines)],
                Unwrite(_lines[-1], suspend_mobject_updating=False),
                FadeIn(hub, suspend_mobject_updating=False),
                *[ReplacementTransform(_dots[i], d, suspend_mobject_updating=False) for i, d in enumerate(dots)],
                Unwrite(_dots[-1], suspend_mobject_updating=False),
                run_time=_t,
                rate_func=linear,
            )
            if n<10:
                self.wait(2*cycle-_t)
            else:
                self.wait(1*cycle-_t)
            _outer = outer
            _lines = lines
            _dots = dots

        self.remove(*_dots, *_lines, _outer)
        self.add(brand)
        self.play(brand.animate.scale(2).center(), FadeOut(hub), run_time=1)
        self.wait()

In [ ]:
%%manim -qm -v WARNING --disable_caching Music5
################################################################ 精度过低
# from manim import *
# import numpy as np
import math

config.frame_width=9
config.frame_height=9

config.frame_size=(360,360)
config.frame_rate=60

# config.frame_size=(1080,1920)
# config.frame_rate=60

# config.frame_size=(2560,1440)
# config.frame_rate=60

def get_points_from_curve(vmob, num=200):
    return [
        vmob.point_from_proportion(perc)
        for perc in np.linspace(0, 1, num)
    ]

def get_threshold(dot, curve):
    threshold = dot.radius
    if dot.get_stroke_opacity() > 0:
        threshold += dot.get_stroke_width()/100
    if curve.get_stroke_opacity() > 0:
        threshold += curve.get_stroke_width()/200
    return threshold**2
def hit_at(dp, curve_points, threshold):
    cp_list = []
    dist_list = []
    for cp in curve_points:
        dist = (dp[0]-cp[0])**2 + (dp[1]-cp[1])**2
        if dist <= threshold:
            cp_list.append(cp)
            dist_list.append(dist)
    if cp_list:
        # i = len(cp_list) // 2
        i = dist_list.index(min(dist_list))
        return cp_list[i]
    return

def bounce_to(dot, curve, hit_point, direction):
    d = dot.get_center() - hit_point
    tangent_line = TangentLine(curve, curve.proportion_from_point(hit_point))
    tangent_vector = tangent_line.get_vector()
    normal_vector = np.cross(OUT, tangent_vector)
    # projection_vector = np.dot(d, normal_vector) * normal_vector
    # symmetric_vector = projection_vector * 2 - d
    # symmetric_dot = symmetric_vector + hit_point
    new_direction = np.dot(-direction, normal_vector) * normal_vector * 2 + direction
    return new_direction

class Dot_(Dot):
    def __init__(self,
        speed=1,
        direction=RIGHT,
        point=ORIGIN, radius=DEFAULT_DOT_RADIUS, stroke_width=0, fill_opacity=1, color=WHITE, **kwargs
    ):
        super().__init__(point, radius, stroke_width, fill_opacity, color, **kwargs)
        self._s = speed
        self._d = direction

    def _update_with(self, c, scene, func=lambda _:None):
        d = self
        cps = get_points_from_curve(c)
        threshold = (1/config.frame_rate * d._s)**2
        def update_d(d, dt):
            cp = hit_at(d.get_center(), cps, threshold)
            if cp is not None:
                func(cp)
                scene.add_sound(f'vibraphone_c/{d._i}.wav', -.1, -10)
                d._d = bounce_to(d, c, cp, d._d)
                d.shift(dt * d._s * d._d * 1)
                if hit_at(d.get_center(), cps, threshold) is not None:
                    d.shift(dt * d._s * d._d * 1)
            else:
                next_pos = d.get_center() + (dt * d._s * d._d)
                d.move_to(next_pos)
        d.add_updater(update_d)

class Music5(Scene):
    def construct(self):
        n = 4
        c_ = [RED_D, GOLD, TEAL_D, BLUE_D]
        colors = color_gradient(c_, n)


        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).scale(.6).next_to(logo, aligned_edge=DOWN, buff=0)
        brand = VGroup(logo, watermark).center()
        self.play(brand.animate(run_time=.5).scale(.5).to_edge(DL))

        ellipse = Ellipse(10, 6, color=WHITE)
        rx = math.sqrt(5**2 - 3**2)
        ld = Dot([-rx,0,0], .15)
        rd = Dot([rx,0,0],  .15)

        pencil = SVGMobject('pics/pencil.svg').set_color(WHITE)
        pencil.height = .5
        pencil._p = .25
        def update_pencil(m, dt):
            m.align_to(ellipse.point_from_proportion(m._p), DL)
            m._p = (m._p - dt/3) % 1
        pencil.add_updater(update_pencil)
        pencil.update(0)

        path = VMobject()
        path.set_points_as_corners([pencil.get_corner(DL), pencil.get_corner(DL)])
        def update_path(path):
            previous_path = path.copy()
            previous_path.add_points_as_corners([pencil.get_corner(DL)])
            path.become(previous_path)
        path.add_updater(update_path)

        def get_links():
            l1 = DashedLine(ld.get_center(), pencil.get_corner(DL))
            l2 = DashedLine(rd.get_center(), pencil.get_corner(DL))
            return VGroup(l1, l2)
        links = always_redraw(get_links)

        self.add_sound('sound_effects/pencil.wav', -.1, -5)
        self.add(links, path, pencil, ld, rd)
        self.wait(3.05)
        self.remove(pencil, links)
        path.remove_updater(update_path)
        self.wait()


        directions = Circle()
        dots = [Dot_(color=colors[i]).move_to(ld) for i in range(n)]
        for i, p in enumerate(np.arange(1/n/2, 1, 1/n)):
            dots[i]._s=2
            dots[i]._d=directions.point_from_proportion(p)
            dots[i]._i = i
            dots[i]._update_with(ellipse, self)
            dots[i]._trace = TracedPath(dots[i].get_center, stroke_opacity=.5, stroke_width=2, stroke_color=colors[i])#, dissipating_time=0.2
        self.add(*[d._trace for d in dots], *dots)
        self.wait(5)
        self.play(Unwrite(ld), Unwrite(rd), run_time=1, suspend_mobject_updating=False)
        self.wait(5)


        self.remove(*dots)
        self.play(brand.animate(run_time=.5).scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching Music4

config.frame_width=16
config.frame_height=9

# config.frame_size=(360,640)
# config.frame_rate=15

# config.frame_size=(1080,1920)
# config.frame_rate=60

config.frame_size=(2560,1440)
config.frame_rate=60

class Dot_(Dot):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._o = 0
        self._od = 0
        self.set_opacity(0)
        self.add_updater(self._update_opacity)

    def _update_opacity(self, m, dt):
        if m._o <= 0:
            pass
        elif m._od == 1:
            m._o += dt * 3
        elif m._od == -1:
            m._o -= dt / 4
        o = np.clip(m._o, 0, 1)
        m.set_fill(opacity=o)
        if o >= 1:
            m._od = -1
        if o < 0:
            m._od = 1
    def _set_o(self, o, od=1):
        self._o = o
        self._od = od
        self.set_stroke(opacity=1)


class Music4(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(
            lambda _, dt: None
        )
        brand = VGroup(logo, watermark)
        brand.scale(.5).next_to(DOWN*4.5, UP, buff=0)
        self.add(brand)

        n = 6
        c_ = [RED_D, GOLD, TEAL_D, BLUE_D]
        speed_factor = 1/2.4
        dot_radius = .2
        radius_list = np.linspace(1, 3.7, n)
        colors = color_gradient(c_, n)
        beats = [
            [0, 4, 8, 12], #kick
            [4, 12], #clap
            [2, 10], #hat
            [2, 6, 10, 14], #shake
            [4,12], #whistle
            [6, 14], #bell
        ]
        instruments = [2,4,6,13,'whistle',1]

        music = SVGMobject('pics/music.svg').set_color(WHITE)
        music.height = 1

        line1 = Line(UP, [0,4,0]).set_stroke(WHITE, 20, .8)
        line1.add_updater(lambda _, dt: None)
        line2 = DashedLine(RIGHT, [7,0,0])
        line3 = DashedLine(DOWN, [0,-4,0])
        line4 = DashedLine(LEFT, [-7,0,0])
        lines = VGroup(line1,line2,line3,line4).set_opacity(.8)

        traces = VGroup(*[
            # Circle(r, stroke_width=2).rotate(PI/2).set_color(c)
            Square(r*2/1.414, stroke_width=2).rotate(PI/4).set_color(c)
            for r, c in zip(radius_list, colors)
        ])


        dots = []
        for i, bs, a, c, r in zip(range(n), beats, traces, colors, instruments):
            ds = []
            for b in bs:
                p = b / 16
                d = Dot_(color=c, radius=dot_radius).set_stroke(WHITE, 2)
                d.move_to(a.point_from_proportion(p))
                d._p = p
                d._c = c
                d._i = i
                d._a = a
                d._r = r
                d._v = False
                ds.append(d)
            dots.append(ds)

        self.add(lines, music)
        self.add_sound('vibraphone_c/0.wav', 0, -10)
        self.play(FadeIn(traces), run_time=.6, lag_ratio=1, rate_func=rate_functions.ease_out_circ)
        # self.play(
        #     DrawBorderThenFill(VGroup(*[d for ds in dots for d in ds])),
        #     rate_func=linear,
        #     lag_ratio=.2,
        #     run_time=1,
        # )

        music.add_updater(lambda m, dt: m.rotate(-dt*speed_factor*TAU))

        self.time=0
        def timer(dt):
            self.time+=dt
        self.add_updater(timer)

        def tfunc(t):
            t = t % 1
            return 1-t

        # def time2perc():
        #     return self.time * speed_factor

        for ds in dots:
            for d in ds:
                def dot_updater(m, dt):
                    m.move_to(m._a.point_from_proportion(m._p))
                    p = (m._p - dt*speed_factor) % 1
                    if m._v and (p - m._p > 0):
                        line1.set_color(m._c)
                        music.set_color(m._c)
                        watermark.set_color(colors[m._i])
                        m._set_o(.8)
                        self.add_sound(f'instruments/{m._r}.wav', 0, 5)
                    m._p = p
                    #
                    # music.set_color(colors[m._i])
                d.add_updater(dot_updater)

        self.wait()
        self.add(*[d for ds in dots for d in ds])
        for i in range(n):
            for d in dots[i]:
                d._v = True
            self.wait(2/speed_factor)

        self.wait(12/speed_factor)

        self.remove(*[d for ds in dots for d in ds])
        self.add(brand)
        self.play(brand.animate.scale(2).center())
        self.wait()

In [ ]:
%%manim -qm -v WARNING --disable_caching Music3

import math

class Sound(VGroup):
    def __init__(self, c=WHITE, height=.4, *vmobs, **kwargs):
        super().__init__(*vmobs, **kwargs)
        self._o = 0
        self._d = 0
        self._c = c
        self.arc1 = Arc(height/3, PI/4, PI/2).set_color(c)
        self.arc2 = Arc(height/3*2, PI/4, PI/2).set_color(c)
        self.arc3 = Arc(height, PI/4, PI/2).set_color(c)
        self.add(self.arc1, self.arc2, self.arc3)
        self.add_updater(self._update_opacity)

    def _update_opacity(self, m, dt):
        if self._o <= 0:
            pass
        elif m._d == 1:
            self._o += dt * 3
        elif m._d == -1:
            self._o -= dt * 2
        o1 = np.clip(self._o, 0, 1)
        o2 = np.clip(self._o-.2, 0, 1)
        o3 = np.clip(self._o-.5, 0, 1)
        self.arc1.set_stroke(opacity=o1)
        self.arc2.set_stroke(opacity=o2)
        self.arc3.set_stroke(opacity=o3)
        if o3 >= 1:
            m._d = -1
        if o1 < 0:
            m._d = 1
    def _set_o(self, o):
        self._o = o
        self._d = 1

class Music3(Scene):
    def construct(self):
        logo = SVGMobject('logo.svg')
        logo.height = 2
        # logo = Square().set_fill(WHITE,1)
        watermark = Text('PccFree.Space').next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        self.play(Write(brand, run_time=2, rate_func=linear))
        self.play(brand.animate.scale(.5).to_edge(DL))

        n = 14
        c_ = [RED_D, GOLD, TEAL_D, BLUE_D]
        speed_factor = 1/19
        dot_radius = .25
        radius_list = np.linspace(-5, 5, n)
        colors = color_gradient(c_, n)


        line1 = Line(LEFT*7, RIGHT*7, stroke_width=8).set_opacity(.8)
        # line2 = Line(LEFT*7, RIGHT*7, stroke_width=6).shift(DOWN*3)
        lines = VGroup(line1)

        traces = VGroup(*[
            Line([r,3-dot_radius,0], [r,dot_radius-3,0], stroke_width=56).set_opacity(.4).set_color(c)
            for r, c in zip(radius_list, colors)
        ])

        dots = VGroup(*[
            Dot(color=c, radius=dot_radius).set_stroke(WHITE, 2).set_opacity(.9).move_to(a.points[0])
            # Square(dot_radius*2).set_stroke(WHITE, 2).set_fill(c,1).move_to(a.points[0])
            for a, c in zip(traces, colors)
        ])

        sounds = VGroup(*[
            Sound(c=c).next_to(d, UP, buff=0) for d, c in zip(dots, colors)
        ])

        self.play(
            DrawBorderThenFill(traces),
            Create(lines),
            run_time=2,
            lag_ratio=.6,
            rate_func=linear,
        )

        self.play(
            DrawBorderThenFill(dots),
            run_time=1,
            rate_func=linear,
            lag_ratio=.2,
        )

        self.add(sounds)

        self.time=0
        def timer(_, dt):
            self.time+=dt
        brand.add_updater(timer)

        def tfunc(t):
            return math.sin(t)**2
            # t = t % 2
            # if t < 1:
            #     return t
            # return 2-t

        def time2perc(m):
            return tfunc( (19-m._i) *self.time * speed_factor)

        for d, a, r, i in zip(dots, traces, radius_list, range(n)):
            d._i = i
            d._a = a
            d._r = r
            d._p = 0
            d._d = 1
            def dot_updater(m):
                p = time2perc(m)
                m.move_to(m._a.point_from_proportion(p))
                delta_p = (p-.5) * (m._p-.5)
                m._p = p
                if delta_p <= 0:
                    watermark.set_color(colors[m._i])
                    lines.set_color(colors[m._i])
                    sounds[m._i]._set_o(.5)
                    self.add_sound(f'vibraphone_c/{15-m._i}.wav', -.1, -10)
                    # m._d *= -1
            d.add_updater(dot_updater)

        for i in range(1):
            self.wait(30)
        self.play(FadeOut(lines), FadeOut(dots))
        self.play(brand.animate.scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching Music2

config.frame_width=16
config.frame_height=9

# config.frame_size=(480,480)
# config.frame_rate=5

# config.frame_size=(1080,1920)
# config.frame_rate=60

config.frame_size=(2560,1440)
config.frame_rate=60

class Light(VGroup):
    def __init__(self, filepath, hight=1, *vmobjects, **kwargs):
        super().__init__(*vmobjects, **kwargs)
        self.svg = SVGMobject(filepath)
        self.svg.height = hight
        # [2, 5]
        self.svg[1].set_opacity(.7)
        self.svg[4].set_opacity(.9)
        self.bulb = Arc(.25*hight,-PI,PI).set_fill(WHITE, 1).move_to(self.svg[3])
        self.add(self.svg, self.bulb)
        self.turn_off()
        self.add_updater(lambda _, dt: None)
    def _color(self, c):
        self.bulb.set_color(c)
        for i in [0,1,3]:
            self.svg[i].set_color(c)
        return self
    def turn_on(self):
        self.svg[0].set_opacity(1)
        self.bulb.set_opacity(1)
    def turn_off(self):
        self.svg[0].set_opacity(0)
        self.bulb.set_opacity(0)
    def _top(self):
        return self.svg[4].get_top()

class Dot_(Dot):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._o = 1
        self._d = 0
        self.add_updater(self._update_opacity)

    def _update_opacity(self, m, dt):
        if m._o <= 0:
            pass
        elif m._d == 1:
            m._o += dt * 3
        elif m._d == -1:
            m._o -= dt / 3
        o = np.clip(m._o, 0, 1)
        m.set_fill(opacity=o)
        if o >= 1:
            m._d = -1
        if o < 0:
            m._d = 1
    def _set_o(self, o):
        self._o = o
        self._d = 1

class _Tap(Rectangle):
    def __init__(
        self,
        color = WHITE,
        height = 1,
        width = .5,
        grid_xstep = None,
        grid_ystep = None,
        mark_paths_closed = True,
        close_new_points = True,
        **kwargs
    ):
        super().__init__(color, height, width, grid_xstep, grid_ystep, mark_paths_closed, close_new_points, **kwargs)
        self._o = 1
        self.add_updater(self._update_opacity)
    def _update_opacity(self, m, dt):
        if m._o >= 1: pass
        else: m._o += dt/3
        o = np.clip(m._o, 0, 1)
        m.set_fill(opacity=o)
    def _set_o(self, o):
        self._o = o

class Music2(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center().set_z_index(999)
        self.play(brand.animate(run_time=.5).scale(.5).next_to(DOWN*4.5, UP, buff=0))

        n = 16
        speed_factor = 1/180
        dot_radius = .1
        radius_list = np.linspace(.8, 3.8, n)
        c_ = [RED_D, GOLD, TEAL_D, BLUE_D]
        colors = color_gradient(c_, n)

        light = Light('pics/light.svg', .8)

        # line1 = Line(UP*4, light._top(), stroke_width=4)
        # line2 = Line([-7,4,0], [7,4,0])
        # lines = VGroup(line1)

        traces = VGroup(*[
            Circle(r, stroke_width=3).rotate(PI/2).set_color(c)
            for r, c in zip(radius_list, colors)
        ])

        dots = VGroup(*[
            Dot_(color=c, radius=dot_radius).set_stroke(c, 3).move_to(a.points[0])
            for a, c in zip(traces, colors)
        ])

        taps_up = VGroup(*[
            _Tap(colors[i], 1/6, 1/6).set_stroke(BLACK,4,.8).shift(UP*radius_list[i])
            for i in range(n)
        ])
        taps_left = taps_up.copy().rotate(PI/2, about_point=ORIGIN)
        taps_down = taps_left.copy().rotate(PI/2, about_point=ORIGIN)
        taps_right = taps_down.copy().rotate(PI/2, about_point=ORIGIN)
        taps = VGroup(taps_up, taps_left, taps_down, taps_right)

        self.add(light)
        self.wait(.1)
        self.add_sound('vibraphone_c/1.wav', 0, -10)
        light.turn_on()
        self.play(
            FadeIn(traces),
            FadeIn(taps),
            run_time=.5,
            lag_ratio=.8,
            rate_func=linear
        )
        self.play(
            Create(dots),
            rate_func=linear,
            lag_ratio=.5,
            run_time=.4,
        )

        self.time=0
        def timer(_, dt):
            self.time+=dt
        brand.add_updater(timer)

        def tfunc(t):
            t = t % 1
            return 1-t

        def time2perc(m):
            return tfunc( (25-m._i) * self.time * speed_factor)

        for d, a, r, i in zip(dots, traces, radius_list, range(n)):
            d._i = i
            d._a = a
            d._r = r
            d._p = 1
            def dot_updater(m):
                m.move_to(m._a.point_from_proportion(m._p))
                p = time2perc(m)
                delta_p = p - m._p
                if delta_p > 0:
                    watermark.set_color(colors[m._i])
                    light._color(colors[m._i])
                    m._set_o(.9)
                    taps_up[m._i]._set_o(.2)
                    self.add_sound(f'kalimba_f/{15-m._i}.wav', -.0, -10)
                elif p<=.75 and m._p>=.75:
                    watermark.set_color(colors[m._i])
                    light._color(colors[m._i])
                    m._set_o(.9)
                    taps_right[m._i]._set_o(.2)
                    self.add_sound(f'kalimba_f/{15-m._i}.wav', -.0, -10)
                elif p<=.5 and m._p>=.5:
                    watermark.set_color(colors[m._i])
                    light._color(colors[m._i])
                    m._set_o(.9)
                    taps_down[m._i]._set_o(.2)
                    self.add_sound(f'kalimba_f/{15-m._i}.wav', -.0, -10)
                elif p<=.25 and m._p>=.25:
                    watermark.set_color(colors[m._i])
                    light._color(colors[m._i])
                    m._set_o(.9)
                    taps_left[m._i]._set_o(.2)
                    self.add_sound(f'kalimba_f/{15-m._i}.wav', -.0, -10)
                m._p = p
            d.add_updater(dot_updater)

        self.wait(180.03)

        self.remove(dots, light)
        self.play(brand.animate.scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching Music1

config.frame_width=16
config.frame_height=9

# config.frame_size=(360,360)
# config.frame_rate=60

# config.frame_size=(1080,1920)
# config.frame_rate=60

config.frame_size=(2560,1440)
config.frame_rate=60

class Dot_(Dot):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._o = 1
        self._d = 0
        self.add_updater(self._update_opacity)

    def _update_opacity(self, m, dt):
        if m._o <= 0:
            pass
        elif m._d == 1:
            m._o += dt * 3
        elif m._d == -1:
            m._o -= dt / 3
        o = np.clip(m._o, 0, 1)
        m.set_fill(opacity=o)
        if o >= 1:
            m._d = -1
        if o < 0:
            m._d = 1
    def _set_o(self, o):
        self._o = o
        self._d = 1

class Music1(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        self.add(brand)
        self.play(brand.animate(run_time=.8, rate_func=linear).scale(.5).next_to(DOWN*4.5, UP, buff=0))

        n = 16
        # speed_factor = 1/42
        speed_factor = 1/48
        dot_radius = .25
        radius_list = np.linspace(4, 7, n)
        c_ = [RED_D, GOLD, TEAL, BLUE_D]
        colors = color_gradient(c_, n)
        # colors = [
        #     '#30f2f2',
        #     '#ffd23f',
        #     '#ff6f59',
        #     '#6290c3',
        #     '#a0a4b8',
        #     '#ff9b85',
        #     '#ffd97d',
        #     '#aaf683',
        #     '#60d394',
        #     '#ee6055',
        #     '#f2d0a9',
        #     '#f1e3d3',
        #     '#99c1b9',
        #     '#8e7dbe',
        #     '#00bbf9',
        #     '#00f5d4',
        # ]

        line1 = Line(UP*4, DOWN*4+LEFT*4, stroke_width=10)
        line2 = Line(UP*4, DOWN*4+RIGHT*4, stroke_width=10)
        lines = VGroup(line1, line2)

        angles = VGroup(*[
            Angle(line1, line2, r, quadrant=(1,1))
            for r in radius_list
        ]).shift(DOWN*(dot_radius+.1)*1.414)

        dots = VGroup(*[
            Dot_(color=c, radius=dot_radius).set_stroke(WHITE, 2).move_to(a.points[0])
            for a, c in zip(angles, colors)
        ])

        def get_links():
            return VGroup(*[
                Line(UP*4, d.get_center()).set_stroke(WHITE, 6, .4)
                for d, c in zip(dots, colors)
            ])
        links = always_redraw(get_links)

        self.add(lines)
        self.play(
            Create(links),
            DrawBorderThenFill(dots),
            run_time=.2,
            rate_func=linear,
            lag_ratio=.2,
            suspend_mobject_updating=False,
        )

        self.time=0
        def timer(_, dt):
            self.time+=dt
        brand.add_updater(timer)

        def tfunc(t):
            t = t % 2
            if t < 1:
                return t
            return 2-t

        def time2perc(m):
            return tfunc((30-m._i) *self.time * speed_factor)

        for d, a, r, i in zip(dots, angles, radius_list, range(n)):
            d._i = i
            d._a = a
            d._r = r
            d._p = 0
            d._md = 1
            def dot_updater(m):
                p = time2perc(m)
                m.move_to(m._a.point_from_proportion(p))
                delta_p = p - m._p
                m._p = p
                if delta_p * m._md < 0:
                    watermark.set_color(colors[m._i])
                    m._set_o(.7)
                    if m._md == 1:
                        line2.set_color(colors[m._i])
                    else:
                        line1.set_color(colors[m._i])
                    self.add_sound(f'vibraphone_c/{15-m._i}.wav', -.1, -10)
                    m._md *= -1
            d.add_updater(dot_updater)

        # self.wait(84.07)
        self.wait(96.03)

        self.remove(lines, links, dots)
        self.play(brand.animate.scale(2).center())
        self.wait()

In [ ]:
%%manim -v WARNING --disable_caching Music0

config.frame_width=16
config.frame_height=9

# config.frame_size=(480,480)
# config.frame_rate=60

# config.frame_size=(1080,1920)
# config.frame_rate=60

config.frame_size=(2560,1440)
config.frame_rate=60

class Dot_(Dot):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._o = 1
        self._d = 0
        self.add_updater(self._update_opacity)

    def _update_opacity(self, m, dt):
        if m._o <= 0:
            pass
        elif m._d == 1:
            m._o += dt * 3
        elif m._d == -1:
            m._o -= dt / 4
        o = np.clip(m._o, 0, 1)
        m.set_fill(opacity=o)
        if o >= 1:
            m._d = -1
        if o < 0:
            m._d = 1
    def _set_o(self, o):
        self._o = o
        self._d = 1

class Music0(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="sans-serif", weight=BOLD).next_to(logo, aligned_edge=DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark).center()
        self.add(brand)
        self.play(brand.animate(run_time=.8, rate_func=linear).scale(.5).next_to(DOWN*4.5, UP, buff=0))

        n = 16
        # speed_factor = 1/42
        speed_factor = 1/42
        dot_radius = .2
        radius_list = np.linspace(2, 7, n)
        c_ = [RED_D, GOLD, TEAL, BLUE_D]
        colors = color_gradient(c_, n)

        line1 = Line(DOWN*4, UR*4, stroke_width=10)
        line2 = Line(DOWN*4, UL*4, stroke_width=10)
        lines = VGroup(line1, line2)

        angles = VGroup(*[
            Angle(line1, line2, r, quadrant=(1,1))
            for r in radius_list
        ]).shift(UP*(dot_radius+.1)*1.414)

        dots = VGroup(*[
            Dot(color=c, radius=dot_radius).set_stroke(WHITE, 2).move_to(a.points[0])
            for a, c in zip(angles, colors)
        ])

        # def get_links():
        #     return VGroup(*[
        #         # Line(UP*4, d.get_center()).set_stroke(WHITE, 6, .4)
        #         Line(dots[i].get_center(), dots[i+1].get_center())#.set_stroke(WHITE, 8, .8)
        #         for i in range(n-1)
        #     ])
        # links = always_redraw(get_links)

        self.add(lines)
        self.play(
            # Create(links),
            DrawBorderThenFill(dots),
            run_time=.2,
            rate_func=linear,
            lag_ratio=.2,
            suspend_mobject_updating=False,
        )

        self.time=0
        def timer(_, dt):
            self.time+=dt
        brand.add_updater(timer)

        def tfunc(t):
            t = t % 2
            if t < 1:
                return t
            return 2-t

        def time2perc(m):
            return tfunc((14+m._i) *self.time * speed_factor)

        for d, a, r, i in zip(dots, angles, radius_list, range(n)):
            d._i = i
            d._a = a
            d._r = r
            d._p = 0
            d._md = 1
            def dot_updater(m):
                p = time2perc(m)
                m.move_to(m._a.point_from_proportion(p))
                delta_p = p - m._p
                m._p = p
                if delta_p * m._md < 0:
                    watermark.set_color(colors[m._i])
                    # m._set_o(.7)
                    if m._md == 1:
                        line2.set_color(colors[m._i])
                    else:
                        line1.set_color(colors[m._i])
                    self.add_sound(f'vibraphone_c/{m._i}.wav', 0, -10)
                    m._md *= -1
            d.add_updater(dot_updater)

        self.wait(84.02)

        self.remove(lines, dots)#, links
        self.play(brand.animate.scale(2).center())
        self.wait()